In [21]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt



In [15]:
model_HS = tf.keras.Sequential([
    tf.keras.layers.LSTM(
        units=64,
        return_sequences=True,  # 必須設為 True 來保持序列長度
        input_shape=(50, 6),    # 從原始配置中的 batch_input_shape
        dropout=0.2,
        recurrent_dropout=0.1,
        name='lstm_2'           # 使用相同的層名稱
    ),
    tf.keras.layers.Dense(
        units=1,
        name='dense_2'
    ),
    tf.keras.layers.Activation(
        'sigmoid',
        name='activation_2'
    )
])

# 編譯模型
model_HS.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 打印模型摘要以驗證結構
model_HS.summary()

# 嘗試載入權重
try:
    model_HS.load_weights(r"C:\Users\m2023\Desktop\vision\predict_model\model_weights.h5")
    print("權重載入成功！")
except Exception as e:
    print("載入權重時出錯：", e)

c:\Users\m2023\Anaconda2\envs\Predict_HS\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 50, 64)         │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50, 1)          │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 50, 1)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,241 (71.25 KB)

 Trainable params: 18,241 (71.25 KB)

 Non-trainable params: 0 (0.00 B)

權重載入成功！


In [22]:
model = tf.keras.models.load_model(r"C:\Users\m2023\Downloads\model_converted.h5", compile=False)

c:\Users\m2023\Anaconda2\envs\Predict_HS\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, None, 50, 6)

In [16]:

data_name = r"C:\Users\m2023\Downloads\_16_predict.xlsx"
data = pd.read_excel(data_name)
predict_data = data.iloc[:, 0:7].values  # 使用 iloc 擷取，並轉換為 NumPy 陣列
print(predict_data.shape)

# 2. 創建時間序列數據集
def create_sequences(data, lookback):
    X = []
    for i in range(len(data) - lookback  + 1):
        X.append(data[i:(i + lookback), :]) 
    return np.array(X)

#set parameter
HeelStrike_Detect_Time=np.array([[1]])
tl=0
tt=0
t_before_write=0
t_after_write=0
getdata=1
samplingtime=0.045
detect_state=1
hs_0=0
hs_1=0
i2_1=0
i2_0=0
pelvic_angle=np.array([[100]])
rotation_amplitude=np.array([[200]])
paretic_side=0 #Paretic side setting
hs_index_last=0
hs_index_now=0
pelvic_rotation_thres=12 # init third rule threshold
TR_trigger_num=0

#test
data_5610_2 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_5617_2 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_7750 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_7770 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])
data_11141 = np.array([[1,2,3,4,5,6,7,8,9,10,11]])

data_www = np.zeros((50,6))
block_count = 0
block_time = 0.3

aaaa=0
bbbb = np.array([[1,2,3,4]])
cccc = np.array([[1]])

lookback = 50

print("Initialized")

predict_data = create_sequences(predict_data, lookback)
predict_data.shape

(7913, 7)
Initialized


(7864, 50, 7)

In [17]:
def main():
    i2_1 = 0
    global block_count
    HeelStrike_Detect_Time = np.array([[1]])
    for i in range(len(data) - lookback  + 1):

        if block_count*samplingtime>block_time*0:
            block_count = 0
            print(i)
            input=preprocessing.scale(predict_data[i,:,1:7])
            input = input.reshape(1,50,6)
            hs=model_HS.predict(input)
            
            if hs[0,49,0]>0.015 and predict_data[i,49,5]*180/3.14<-10:
                i1_1=i2_1
                i2_1=predict_data[i,49,0]
                if i2_1-i1_1>0.1:

                    HeelStrike_Detect_Time = np.vstack([HeelStrike_Detect_Time, i2_1])
                    print(HeelStrike_Detect_Time)
        
        block_count+=1 

    return HeelStrike_Detect_Time


def SaveData(HeelStrike_Detect_Time):
    index = len(HeelStrike_Detect_Time)
    index_0=np.array([["rightfoot"]])
    ee = np.vstack([index_0,HeelStrike_Detect_Time[1:index]])
    gg = np.hstack([ee])
    gg_csv = pd.DataFrame(gg)
    gg_csv.to_csv(data_name+'2hstime.csv',header=0,index=0)


In [18]:
HeelStrike_Detect_Time = main()
SaveData(HeelStrike_Detect_Time)

1
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


AttributeError: in user code:

    File "C:\Users\m2023\AppData\Local\Temp\ipykernel_9484\3106129146.py", line 13, in main  *
        hs=model_HS.predict(input)
    File "c:\Users\m2023\Anaconda2\envs\Predict_HS\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\m2023\Anaconda2\envs\Predict_HS\lib\site-packages\optree\ops.py", line 752, in tree_map
        return treespec.unflatten(map(func, *flat_args))

    AttributeError: 'SymbolicTensor' object has no attribute 'numpy'
